In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.optim import lr_scheduler

import pandas as pd
from datetime import datetime
from matplotlib import pyplot as plt
import copy
import argparse
import pandas as pd
from tqdm import tqdm
import torchvision.utils as vutils
import torchvision.transforms as transforms
from datetime import datetime
import os
from moviepy.editor import VideoFileClip
import math
import json
from PIL import Image
from dataloader import getDataLoader
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
import cv2 as cv2
from torch.autograd import Variable

In [50]:
net = models.resnet101(pretrained=True)
# old_conv1 = net.conv1
# net.conv1 = nn.Conv2d(
#     in_channels=6,
#     out_channels=old_conv1.out_channels,
#     kernel_size=old_conv1.kernel_size,
#     stride=old_conv1.stride,
#     padding=old_conv1.padding,
#     bias=old_conv1.bias,
# )
in_channel = net.fc.in_features
net.fc = nn.Sequential(nn.Linear(in_channel, 1024), nn.ReLU(inplace=True),
                        nn.Dropout(0.5), nn.Linear(1024, 512),
                        nn.ReLU(inplace=True), nn.Dropout(0.5),
                        nn.Linear(512, 256), nn.ReLU(inplace=True),
                        nn.Dropout(0.5), nn.Linear(256, 7))
net.bn = nn.BatchNorm1d(num_features=7)
emotion_model_path = '/workspace/chi149/MELD/resnet/emotion2023-05-24 11:36:37/best.pth'
model_data = torch.load(emotion_model_path)
net.load_state_dict(model_data)
net.to('cuda')

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [11]:
file_list = os.listdir('/workspace/chi149/MELD/MELD.Raw/test/test_pic1')
file_list.sort()

In [67]:
trans_test = transforms.Compose([
        transforms.ToTensor(),  # 转换为张量
        transforms.RandomRotation(10),  # 随机旋转（-10到+10度之间）
        transforms.ColorJitter(brightness=0.4,
                               contrast=0.4,
                               saturation=0.4,
                               hue=0.1),  # 随机调整亮度、对比度、饱和度和色调
        
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),  # 标准化图像张量
    ])
test_data_df = pd.read_pickle('/home/chi149/Desktop/MELD/MELD.Raw/test/test_tensor1.pkl')
test_data_df.sort_values(by='cur_filename', ascending=True)
print(len(test_data_df))
test_dataloader = getDataLoader(test_data_df, 'Emotion', trans_test, 'test', 32)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

phase = 'test'
for index, (data, labels) in enumerate(test_dataloader):
    X = torch.as_tensor(data).to(device)
    labels = torch.as_tensor(labels).to(device)
    with torch.set_grad_enabled(phase == 'train'):
        y = net(X)
        _, predicted = torch.max(y.data, 1)
        print(predicted)

6031
tensor([6, 6, 2, 1, 6, 6, 2, 6, 6, 4, 1, 3, 0, 6, 6, 3, 3, 5, 3, 6, 0, 4, 5, 0,
        3, 6, 6, 6, 2, 0, 6, 6], device='cuda:0')
tensor([6, 6, 0, 6, 5, 6, 5, 0, 6, 6, 3, 0, 5, 6, 0, 5, 4, 0, 4, 6, 4, 4, 4, 0,
        0, 6, 6, 6, 5, 1, 5, 6], device='cuda:0')
tensor([1, 6, 6, 4, 6, 0, 0, 6, 4, 3, 4, 4, 0, 0, 1, 6, 6, 2, 0, 2, 6, 6, 6, 6,
        5, 1, 6, 3, 0, 3, 6, 2], device='cuda:0')
tensor([6, 0, 3, 6, 5, 6, 6, 6, 1, 4, 3, 6, 3, 6, 2, 5, 1, 4, 6, 6, 1, 3, 1, 2,
        5, 3, 6, 2, 0, 0, 6, 6], device='cuda:0')
tensor([5, 0, 6, 6, 0, 6, 6, 6, 4, 0, 6, 2, 6, 6, 6, 3, 0, 4, 1, 6, 0, 3, 6, 1,
        5, 5, 4, 0, 6, 2, 6, 5], device='cuda:0')
tensor([2, 5, 4, 6, 0, 2, 6, 6, 5, 6, 6, 0, 1, 0, 0, 0, 0, 4, 3, 6, 0, 0, 6, 3,
        6, 6, 6, 0, 1, 0, 5, 3], device='cuda:0')
tensor([5, 5, 1, 6, 6, 3, 6, 4, 2, 0, 6, 3, 6, 6, 6, 3, 4, 6, 4, 5, 0, 0, 3, 6,
        0, 6, 6, 6, 0, 3, 6, 5], device='cuda:0')
tensor([6, 5, 4, 6, 6, 6, 0, 6, 1, 6, 0, 4, 0, 6, 6, 3, 6, 5, 1, 6, 4, 6, 6, 6,
     

In [87]:
a1 = test_data_df[test_data_df['cur_filename']=='dia53_utt0.mp4']

In [92]:
# input_tensor = test_data_df[test_data_df['cur_filename']=='dia53_utt0.mp4']['img'][790]
for index, input_tensor in enumerate(test_data_df['img'].values):
    index = test_data_df['img'].values.index(input_tensor)
    img_tensor = Variable(torch.unsqueeze(input_tensor, dim=0).float(), requires_grad=False).to(device)
    y = net(img_tensor)
    _, predicted = torch.max(y.data, 1)
    # print(predicted)
    if predicted == torch.tensor([6]).to('cuda'):
        print(test_data_df[index])

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [7]:
trans_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # 随机水平翻转
    transforms.RandomRotation(10),  # 随机旋转（-10到+10度之间）
    transforms.ColorJitter(brightness=0.4,
                            contrast=0.4,
                            saturation=0.4,
                            hue=0.1),  # 随机调整亮度、对比度、饱和度和色调
    transforms.ToTensor(),  # 转换为张量
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225]),  # 标准化图像张量
])

trans_test = transforms.Compose([
    transforms.RandomRotation(10),  # 随机旋转（-10到+10度之间）
    transforms.ColorJitter(brightness=0.4,
                            contrast=0.4,
                            saturation=0.4,
                            hue=0.1),  # 随机调整亮度、对比度、饱和度和色调
    transforms.ToTensor(),  # 转换为张量
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225]),  # 标准化图像张量
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_path = '/workspace/chi149/MELD/MELD.Raw/train/train_tensor1.pkl'
test_path = '/workspace/chi149/MELD/MELD.Raw/test/test_tensor_2pic.pkl'

# generate dataloader
train_data_df = pd.read_pickle(train_path)
test_data_df = pd.read_pickle(test_path)
train_speaker_dataloader = getDataLoader(train_data_df, 'Speaker', trans_train,
                                            'train', 32)
test_speaker_dataloader = getDataLoader(test_data_df, 'Speaker', trans_test,
                                        'test', 32)

train_emotion_dataloader = getDataLoader(train_data_df, 'Emotion', trans_train,
                                            'train', 32)
test_emotion_dataloader = getDataLoader(test_data_df, 'Emotion', trans_test,
                                        'test', 32)

speaker_dataloaders = {
    'train': train_speaker_dataloader,
    'test': test_speaker_dataloader
}
emotion_dataloaders = {
    'train': train_emotion_dataloader,
    'test': test_emotion_dataloader
}

neutral     9663
joy         4088
anger       2715
surprise    2315
sadness     1920
disgust      780
fear         646
Name: Emotion, dtype: int64
tensor([0.0001, 0.0002, 0.0004, 0.0004, 0.0005, 0.0013, 0.0015])


In [11]:
emotion_net = models.resnet50(pretrained=True)
emotion_model_path = '/workspace/chi149/MELD/pathway/resnet/emotion2023-05-24 11:36:37/best.pth'
emotion_net.load_state_dict(torch.load(emotion_model_path), False)
emotion_net.fc = nn.Sequential(
    nn.Linear(emotion_net.fc.in_features, 256),
    nn.Sigmoid(),
    nn.Linear(256, 7),
)
emotion_net = emotion_net.to(device)

In [17]:
for index, data in enumerate(emotion_dataloaders['train']):

    (emotion_X, emotion_labels) = data
    print(emotion_X[0].shape)
    input_example = emotion_X[0]
    break

    emotion_X = torch.as_tensor(emotion_X).to(device)
    emotion_labels = torch.as_tensor(emotion_labels).to(device)

    with torch.no_grad():
        out_emotion = emotion_net(emotion_X)
    print(out_emotion)
    break

torch.Size([3, 224, 224])


In [24]:
from torch.autograd import Variable
input_tensor = Variable(torch.unsqueeze(input_example, dim=0).float(), requires_grad=False)
input_tensor = input_tensor.to(device)
emotion_net(input_tensor)

tensor([[ 0.7816,  0.4112, -0.4188, -0.0827,  0.5604, -0.2444, -0.4443]],
       device='cuda:0', grad_fn=<AddmmBackward>)